### Input Request Format : 

#### Meeting Request that contains the information of :
- Meeting Request UUID 
- Time of the Request 
- Sender Email Address
- Attendees Email Addresses 
- Email Subject
- Email Content


```
{
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "09-07-2025T12:34:55",
    "Location": "IIT Mumbai",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Agentic AI Project Status Update",
    "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
}
 ```

-------------------------------

### Output Format : 


#### Output JSON that contains the information of :
- Meeting Request UUID 
- Time of the Request 
- Sender Email Address
- Attendees Email Addresses with Attendees Events
- Email Subject
- Email Content
- Start Time of the event
- End Time of the event 
- Duration 
- MetaData that may contains to & fro communication details

### Output Format : 

```
{
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "09-07-2025T12:34:55",
    "Location": "IIT Mumbai",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "userone.amd@gmail.com",
            "events": [
                {
                    "StartTime": "2025-07-17T10:30:00+05:30",
                    "EndTime": "2025-07-17T11:00:00+05:30",
                    "NumAttendees": 3,
                    "Attendees": [
                        "userone.amd@gmail.com",
                        "usertwo.amd@gmail.com",
                        "userthree.amd@gmail.com"
                    ],
                    "Summary": "Agentic AI Project Status Update"
                }
            ]
        },
        {
            "email": "usertwo.amd@gmail.com",
            "events": [
                {
                    "StartTime": "2025-07-17T10:00:00+05:30",
                    "EndTime": "2025-07-17T10:30:00+05:30",
                    "NumAttendees": 3,
                    "Attendees": [
                        "userone.amd@gmail.com",
                        "usertwo.amd@gmail.com",
                        "userthree.amd@gmail.com"
                    ],
                    "Summary": "Team Meet"
                },
                {
                    "StartTime": "2025-07-17T10:30:00+05:30",
                    "EndTime": "2025-07-17T11:00:00+05:30",
                    "NumAttendees": 3,
                    "Attendees": [
                        "userone.amd@gmail.com",
                        "usertwo.amd@gmail.com",
                        "userthree.amd@gmail.com"
                    ],
                    "Summary": "Agentic AI Project Status Update"
                }
            ]
        },
        {
            "email": "userthree.amd@gmail.com",
            "events": [
                {
                    "StartTime": "2025-07-17T10:00:00+05:30",
                    "EndTime": "2025-07-17T10:30:00+05:30",
                    "NumAttendees": 3,
                    "Attendees": [
                        "userone.amd@gmail.com",
                        "usertwo.amd@gmail.com",
                        "userthree.amd@gmail.com"
                    ],
                    "Summary": "Team Meet"
                },
                {
                    "StartTime": "2025-07-17T13:00:00+05:30",
                    "EndTime": "2025-07-17T14:00:00+05:30",
                    "NumAttendees": 1,
                    "Attendees": [
                        "SELF"
                    ],
                    "Summary": "Lunch with Customers"
                },
                {
                    "StartTime": "2025-07-17T10:30:00+05:30",
                    "EndTime": "2025-07-17T11:00:00+05:30",
                    "NumAttendees": 3,
                    "Attendees": [
                        "userone.amd@gmail.com",
                        "usertwo.amd@gmail.com",
                        "userthree.amd@gmail.com"
                    ],
                    "Summary": "Agentic AI Project Status Update"
                }
            ]
        }
    ],
    "Subject": "Agentic AI Project Status Update",
    "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project.",
    "EventStart": "2025-07-17T10:30:00+05:30",
    "EventEnd": "2025-07-17T11:00:00+05:30",
    "Duration_mins": "30",
    "MetaData": {}
}
```

In [2]:
import json
import uuid
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

# Your provided function (unchanged)
def retrive_calendar_events(user, start, end):
    events_list = []
    token_path = "./Keys/"+user.split("@")[0]+".token"
    user_creds = Credentials.from_authorized_user_file(token_path)
    calendar_service = build("calendar", "v3", credentials=user_creds)
    events_result = calendar_service.events().list(calendarId='primary', timeMin=start,timeMax=end,singleEvents=True,orderBy='startTime').execute()
    events = events_result.get('items')
    
    for event in events : 
        attendee_list = []
        try:
            for attendee in event["attendees"]: ""
            attendee_list.append(attendee['email'])
        except: 
            attendee_list.append("SELF")
        start_time = event["start"]["dateTime"]
        end_time = event["end"]["dateTime"]
        events_list.append(
            {"StartTime" : start_time, 
             "EndTime": end_time, 
             "NumAttendees" :len(set(attendee_list)), 
             "Attendees" : list(set(attendee_list)),
             "Summary" : event["summary"]})
    return events_list

# Function to process JSON file and transform output
async def process_calendar_data(json_file_path):
    try:
        # Read and parse the JSON file
        with open(json_file_path, 'r') as file:
            input_data = json.load(file)

        # Process each user in the JSON file
        all_events = []
        for entry in input_data:
            user = entry.get('user')
            start = entry.get('start')
            end = entry.get('end')

            if not all([user, start, end]):
                print(f"Skipping invalid entry: {entry}")
                continue

            # Call the provided retrieval function
            events = retrive_calendar_events(user, start, end)

            # Transform to new server format
            for event in events:
                all_events.append({
                    "eventId": str(uuid.uuid4()),  # Generate unique ID
                    "userEmail": user,
                    "title": event["Summary"],
                    "start": event["StartTime"],
                    "end": event["EndTime"],
                    "attendeesCount": event["NumAttendees"]
                })

        # Create final output
        output = {
            "events": all_events,
            "totalEvents": len(all_events)
        }

        # Save to new JSON file (optional, for server)
        with open('output_events.json', 'w') as outfile:
            json.dump(output, outfile, indent=2)

        return output

    except FileNotFoundError:
        print(f"Error: JSON file {json_file_path} not found")
        return None
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

# Example usage
if __name__ == "__main__":
    import asyncio
    json_file_path = "input_data.json"
    result = asyncio.run(process_calendar_data(json_file_path))
    if result:
        print(json.dumps(result, indent=2))

RuntimeError: asyncio.run() cannot be called from a running event loop